In [1]:
import pandas  as pd

In [2]:
def labelConvert(s):
    s = s.strip().lower()
    if s == "m":
        return 0
    if s == "b":
        return 1
    return -1

In [6]:
data = pd.read_csv('wdbc.data', header = None, converters={30:labelConvert})

In [14]:
data

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,-1,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,-1,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,-1,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,-1,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,-1,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,-1,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,-1,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,-1,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,-1,0.12400


In [8]:
def split_Train_Test(data):
    import random
    TRAIN_TEST_RATIO = 0.8
    train = []
    test = []
    for d in data:
        if random.random() < TRAIN_TEST_RATIO:
            train.append(d)
        else:
            test.append(d)
    return train, test

In [10]:
def predict(row, weights):
    activation = weights[0]
    for i in range(len(row)-1):
        activation += int(weights[i + 1]) * int(row[i])
    return 1.0 if activation >= 0.0 else 0.0

### We are using an algorithm called gradient descent.

def train_weights(train, l_rate, n_epoch):
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0.0
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            sum_error += error**2
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row)-1):
                weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
        if epoch % 100 == 0:
            print('> epoch = %4d, lrate = %.4f, error =%6.1f' % (epoch, l_rate, sum_error))
    return weights

## Do not worry about the hyperparameters of this algorithm right now.
train, test = split_Train_Test(data.values)
print("Training", len(train))
print("Testing", len(test))
weights = train_weights(train, l_rate=0.001, n_epoch=2000)
print(weights)

Training 436
Testing 133


ValueError: invalid literal for int() with base 10: 'M'

In [15]:
actuals = [1,1,0,0,1,0,1,0,0,1,1]
prediction = [0,1,0,1,1,0,1,0,0,1,1]

In [18]:
def confusionmatrix(actuals, prediction):
    true_postive =  sum([int(a==1 and p==1) for a , p in zip(actuals, prediction)])
    true_negative = sum([int(a==0 and p==0) for a,p in zip(actuals, prediction)])
    false_postive = sum([int(a==0 and p==1) for a,p in zip(actuals,prediction)])
    false_negative =   sum([int(a==1 and p==0) for a, p in zip(actuals,prediction)])
    return true_postive,true_negative,false_postive,false_negative

In [23]:
true_postive,true_negative,false_postive,false_negative = confusionmatrix(actuals, prediction)

In [25]:
Precision = true_postive/(true_postive+false_postive)
Precision

0.8333333333333334

In [26]:
recal = true_postive/(true_postive + false_negative)
recal

0.8333333333333334

In [29]:
mis_rate = 1- recal
mis_rate

0.16666666666666663

In [31]:
accurecy = (true_postive+true_negative)/(true_postive+true_negative+false_postive+false_negative)
accurecy

0.8181818181818182